### **D2APR: Aprendizado de Máquina e Reconhecimento de Padrões** (IFSP, Campinas) <br/>
**Trabalho 1 - Sao Paulo Real Estate**

Aluno: Hugo Martinelli Watanuki

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>. <br/><br/>

## 2. Obtaining the dataset

In [1]:
import pandas as pd

housing = pd.read_csv('./sample_data/sao-paulo-properties-april-2019.csv')

In [2]:
housing.head()

,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,District,Negotiation Type,Property Type,Latitude,Longitude
0,930,220,47,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.543138,-46.479486
1,1000,148,45,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.550239,-46.480718
2,1000,100,48,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.542818,-46.485665
3,1000,200,48,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.547171,-46.483014
4,1300,410,55,2,2,1,1,1,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.525025,-46.482436


### 0. Imports and default settings for plotting

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

## 🛠️ 5. Prepare the Data

We will consider the same two scenarios for **Polynomial Regression** from Sprint #5 in this sprint:
1. Use _only_ the `median_income`.
2. Use _all features_ except those that generated the aggregate features (`total_rooms`, `total_bedrooms`, `population`, `household`).

### 5.1. Load the cleaned training set

Let's consider the training and testing sets already cleaned (sprint #2):
- Drop duplicated instances (no found)
- Drop instances with `housing_median_age` capped at 52
- Drop instances with `median_house_value` capped at 500001.0

In [ ]:
# load the cleaned training set
housing_train = pd.read_csv('./datasets/housing_train_sprint-2.csv')

In [ ]:
housing_train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-121.37,37.06,25.0,474.0,92.0,300.0,104.0,3.8062,340900.0,INLAND
1,-118.39,34.14,19.0,5076.0,1034.0,2021.0,960.0,5.5683,309200.0,<1H OCEAN
2,-122.07,37.41,26.0,1184.0,225.0,815.0,218.0,5.7657,322300.0,NEAR BAY
3,-121.92,36.57,42.0,3944.0,738.0,1374.0,598.0,4.1740,394400.0,NEAR OCEAN
4,-118.36,33.82,36.0,1083.0,187.0,522.0,187.0,5.7765,339500.0,<1H OCEAN


In [ ]:
housing_train.shape

(14857, 10)

### 5.2. Separate the _features_ and the _target outcome_

In [ ]:
# store the target outcome into a numpy array
y_train = housing_train['median_house_value'].values

# overwrite the dataframe with only the features
housing_train = housing_train.drop(columns=['median_house_value'])

### 5.3. Pipelines

For the sake of simplicity, let's include the **Polynomial Regression** objects (`PolynomialFeatures()` + `LinearRegression()`) into our **pipeline**. So, it is no longer _just_ dedicated to preprocessing.

#### **Scenario 1**

In [ ]:
# pipeline for numerical
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

attributes_scenario_1 = ['median_income']

pipeline_scenario_1 = Pipeline([
    ('imputer', SimpleImputer()),   # let's evaluate the mean and median inputation
    ('poly', PolynomialFeatures()),
    ('scaler', RobustScaler())
])

# we will just use the ColumnTransformer because it automaticaly filters the required columns for us before performing the pipeline.
# (name, transformer, columns)
preprocessed_pipeline_scenario_1 = ColumnTransformer([
    ("numerical", pipeline_scenario_1, attributes_scenario_1)
])


# full pipeline: preprocessing + model training/prediction
full_pipeline_scenario_1 = Pipeline([
        ('preprocessing', preprocessed_pipeline_scenario_1),
        ('lin_regression', LinearRegression())
])

#### **Scenario 2**

In [ ]:
#### feature engineering method from the Sprint #4
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

# our 3 new features are based on some the features: totalrooms,
# column index
rooms_col_idx, bedrooms_col_idx, population_col_idx, households_col_idx = 3, 4, 5, 6

class HousingFeatEngineering(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self  # nothing else to do

    def transform(self, X):
        n_rows = X.shape[0]

        # creating the new features
        rooms_per_household = X[:, rooms_col_idx] / X[:, households_col_idx]
        population_per_household = X[:, population_col_idx] / X[:, households_col_idx]
        bedrooms_per_room = X[:, bedrooms_col_idx] / X[:, rooms_col_idx]

        # to concatenate the new array as columns in our feature matrix, we need to reshape them first
        rooms_per_household = rooms_per_household.reshape((n_rows, 1))
        population_per_household = population_per_household.reshape((n_rows, 1))
        bedrooms_per_room = bedrooms_per_room.reshape((n_rows, 1))

        # concatenating the new features into the feature matrix
        X_out = np.hstack((X, rooms_per_household, population_per_household, bedrooms_per_room))

        return X_out

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

# our 3 new features are based on some the features: totalrooms,
# column index
rooms_col_idx, bedrooms_col_idx, population_col_idx, households_col_idx = 3, 4, 5, 6

class DropFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, verbose=False):
        self.verbose = verbose

    def fit(self, X, y=None):
        return self  # nothing else to do

    def transform(self, X):
        X_out = np.delete(X, [rooms_col_idx, bedrooms_col_idx, population_col_idx, households_col_idx], axis=1)

        # for debugging
        if self.verbose:
            np.set_printoptions(suppress=True)
            print('X[:5]')
            print(X[:5])
            print('\nX_out[:5]')
            print(X_out[:5])

        return X_out

In [ ]:
# pipeline for numerical
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

attributes_scenario_2 = housing_train.columns.drop('ocean_proximity')

pipeline_scenario_2 = Pipeline([
    ('imputer', SimpleImputer()),  # let's evaluate the mean and median inputation
    ('feat_engineering', HousingFeatEngineering()),
    ('drop_features', DropFeatures(verbose=False)),
    ('poly', PolynomialFeatures()),
    ('scaler', RobustScaler())
])

# we will just use the ColumnTransformer because it automaticaly filters the required columns for us before performing the pipeline.
# (name, transformer, columns)
preprocessed_pipeline_scenario_2 = ColumnTransformer([
    ("numerical", pipeline_scenario_2, attributes_scenario_2)
])

# full pipeline: preprocessing + model training/prediction
full_pipeline_scenario_2 = Pipeline([
        ('preprocessing', preprocessed_pipeline_scenario_2),
        ('lin_regression', LinearRegression())
])

## 🏋️‍♀️ 6. Train ML Algorithms

### 6.1. Grid-Search (fine-tunning)
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

### **Scenario 1**

#### **Finding out the hyperparameter key names**

In [ ]:
full_pipeline_scenario_1.get_params()

{'memory': None,
 'steps': [('preprocessing',
   ColumnTransformer(transformers=[('numerical',
                                    Pipeline(steps=[('imputer', SimpleImputer()),
                                                    ('poly', PolynomialFeatures()),
                                                    ('scaler', RobustScaler())]),
                                    ['median_income'])])),
  ('lin_regression', LinearRegression())],
 'verbose': False,
 'preprocessing': ColumnTransformer(transformers=[('numerical',
                                  Pipeline(steps=[('imputer', SimpleImputer()),
                                                  ('poly', PolynomialFeatures()),
                                                  ('scaler', RobustScaler())]),
                                  ['median_income'])]),
 'lin_regression': LinearRegression(),
 'preprocessing__n_jobs': None,
 'preprocessing__remainder': 'drop',
 'preprocessing__sparse_threshold': 0.3,
 'preprocessing__transfor

#### **Grid-search**

In [ ]:
from sklearn.model_selection import GridSearchCV

# search space
param_grid_scenario_1 = [
    {
    'preprocessing__numerical__imputer__strategy': ['mean', 'median'],
    'preprocessing__numerical__poly__degree': [2, 3, 10, 20, 30],
    'preprocessing__numerical__poly__include_bias': [False, True],
    'preprocessing__numerical__poly__interaction_only': [False, True]
    }
]

grid_search_scenario_1 = GridSearchCV(full_pipeline_scenario_1, param_grid_scenario_1, cv=5, scoring='neg_mean_squared_error', return_train_score=True, verbose=1)
grid_search_scenario_1.fit(housing_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('numerical',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('poly',
                                                                                          PolynomialFeatures()),
                                                                                         ('scaler',
                                                                                          RobustScaler())]),
                                                                         ['median_income'])])),
                                       ('lin_regression', LinearRegression())]),
             param_grid=[{'prepr

In [ ]:
cvres = grid_search_scenario_1.cv_results_

# (approximated) RMSE
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

71691.10772210758 {'preprocessing__numerical__imputer__strategy': 'mean', 'preprocessing__numerical__poly__degree': 2, 'preprocessing__numerical__poly__include_bias': False, 'preprocessing__numerical__poly__interaction_only': False}
71676.82921015851 {'preprocessing__numerical__imputer__strategy': 'mean', 'preprocessing__numerical__poly__degree': 2, 'preprocessing__numerical__poly__include_bias': False, 'preprocessing__numerical__poly__interaction_only': True}
71691.10772210758 {'preprocessing__numerical__imputer__strategy': 'mean', 'preprocessing__numerical__poly__degree': 2, 'preprocessing__numerical__poly__include_bias': True, 'preprocessing__numerical__poly__interaction_only': False}
71676.82921015852 {'preprocessing__numerical__imputer__strategy': 'mean', 'preprocessing__numerical__poly__degree': 2, 'preprocessing__numerical__poly__include_bias': True, 'preprocessing__numerical__poly__interaction_only': True}
71456.56128983348 {'preprocessing__numerical__imputer__strategy': 'mean'

In [ ]:
# best params
grid_search_scenario_1.best_params_

{'preprocessing__numerical__imputer__strategy': 'mean',
 'preprocessing__numerical__poly__degree': 10,
 'preprocessing__numerical__poly__include_bias': False,
 'preprocessing__numerical__poly__interaction_only': False}

In [ ]:
# best estimator
# if refit=True, it returns a model trained with the full training set with the best hyperparameters
best_model = grid_search_scenario_1.best_estimator_
best_model

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('poly',
                                                                   PolynomialFeatures(degree=10,
                                                                                      include_bias=False)),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['median_income'])])),
                ('lin_regression', LinearRegression())])

In [ ]:
# get the (approximated) RMSE
np.sqrt(-grid_search_scenario_1.best_score_)

71298.70981488981

In [ ]:
# get the approximated RMSE and its standard deviation
best_index = grid_search_scenario_1.best_index_
best_score = np.sqrt(-grid_search_scenario_1.cv_results_['mean_test_score'][best_index])
best_score_std = np.sqrt(grid_search_scenario_1.cv_results_['std_test_score'][best_index])

print(f'Best score: {best_score} +- {best_score_std}')

Best score: 71298.70981488981 +- 11036.55480104328


<br/>

The **cross validation score** from **the best model** (\\$71,298.70 ± \\$11,036.55) is _slightly better_ than the score of Scenario 1 without fine-tunning (\\$71,671.86 ± \$1,479.30) from Sprint #5, its results are more unstable (see its _standard deviation_). <br/>
Moreover, its **cross validation score** keeps being _considerably higher_ than that of `Linear Regression` from the previous Sprint (\\$58,371.04 ± \$1,757.91).

<table align="left" class="dashed-box">
<tr>
    <td>💡</td>
    <td>Mathematically, the correct way to estimate the <b>cross-validation RMSE</b> and its <b>standard deviation</b> is to apply the <i>square root</i> for the each split <i>negative MSE score</i>, and then compute the <i>mean</i> and <i>std</i> with these new scores.</td>
</tr>
<tr>
    <td></td>
    <td>For that, we need to 'debug' the _cross validation results.</td>
</tr>
</table><br/><br/>

In [ ]:
grid_search_scenario_1.cv_results_

{'mean_fit_time': array([0.01887522, 0.0094492 , 0.00991473, 0.01025591, 0.01390386,
        0.01003895, 0.01409917, 0.01014118, 0.01996737, 0.01102452,
        0.02211404, 0.01221519, 0.03636308, 0.0101171 , 0.033359  ,
        0.01285214, 0.04814892, 0.0117455 , 0.04784751, 0.01231089,
        0.01181126, 0.00959897, 0.01091461, 0.01017919, 0.01351895,
        0.00950961, 0.01344018, 0.0112464 , 0.02205029, 0.01144061,
        0.02214503, 0.01309052, 0.04201412, 0.01209021, 0.04594536,
        0.01328597, 0.04784422, 0.01263294, 0.04693222, 0.01277361]),
 'std_fit_time': array([0.00306254, 0.00147188, 0.00059162, 0.00081139, 0.00206498,
        0.00197489, 0.00128092, 0.00131696, 0.00153409, 0.00047527,
        0.00095572, 0.00066597, 0.00206565, 0.00020597, 0.00272837,
        0.00068374, 0.00365625, 0.00205953, 0.00321318, 0.00097735,
        0.00085682, 0.00073027, 0.00043163, 0.00048236, 0.00191204,
        0.00091917, 0.00098501, 0.00108267, 0.00116167, 0.00019569,
        0.001

In [ ]:
n_folds = 5
split_keys = [f'split{i}_test_score' for i in range(n_folds)]
best_index = grid_search_scenario_1.best_index_

rmse_scores = []

for key in split_keys:
    neg_mse_score = grid_search_scenario_1.cv_results_[key][best_index]
    rmse_scores.append(np.sqrt(-neg_mse_score))

best_rmse = np.mean(rmse_scores)
best_rmse_std = np.std(rmse_scores)


print(f'Best RMSE score: {best_rmse} +- {best_rmse_std}')

Best RMSE score: 71293.59154095282 +- 854.2988116844386


By doing it in the _mathematically correct way_, we see that the **true mean RMSE** (\\$71,293.59) is very close from the approximated one (\\$71,298.70). <br/>
However, the **true standard deviation for RMSE** (\$854.29) is _considerably smaller_ than the approximated one (\\$11,036.55).

### **Scenario 2**

In [ ]:
full_pipeline_scenario_2.get_params()

{'memory': None,
 'steps': [('preprocessing',
   ColumnTransformer(transformers=[('numerical',
                                    Pipeline(steps=[('imputer', SimpleImputer()),
                                                    ('feat_engineering',
                                                     HousingFeatEngineering()),
                                                    ('drop_features',
                                                     DropFeatures()),
                                                    ('poly', PolynomialFeatures()),
                                                    ('scaler', RobustScaler())]),
                                    Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
          'total_bedrooms', 'population', 'households', 'median_income'],
         dtype='object'))])),
  ('lin_regression', LinearRegression())],
 'verbose': False,
 'preprocessing': ColumnTransformer(transformers=[('numerical',
                             

Since our feature matrix has a considerable number of features, for computation time reasons, we decided to:
- Consider a small number of hyperparameter combinations (small search space)
- Low polynomial degrees

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid_scenario_2 = [
    {
     'preprocessing__numerical__imputer__strategy': ['mean', 'median'],
     'preprocessing__numerical__poly__degree': [2, 3, 4, 5],
     'preprocessing__numerical__poly__include_bias': [False, True],
     'preprocessing__numerical__poly__interaction_only': [False]
    }
]

grid_search_scenario_2 = GridSearchCV(full_pipeline_scenario_2, param_grid_scenario_2, cv=5, scoring='neg_mean_squared_error', return_train_score=True, verbose=1)
grid_search_scenario_2.fit(housing_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('numerical',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('feat_engineering',
                                                                                          HousingFeatEngineering()),
                                                                                         ('drop_features',
                                                                                          DropFeatures()),
                                                                                         ('poly',
                                                                                          

In [ ]:
cvres = grid_search_scenario_2.cv_results_

for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

626165.2300283933 {'preprocessing__numerical__imputer__strategy': 'mean', 'preprocessing__numerical__poly__degree': 2, 'preprocessing__numerical__poly__include_bias': False, 'preprocessing__numerical__poly__interaction_only': False}
626165.2300291593 {'preprocessing__numerical__imputer__strategy': 'mean', 'preprocessing__numerical__poly__degree': 2, 'preprocessing__numerical__poly__include_bias': True, 'preprocessing__numerical__poly__interaction_only': False}
206771298.9577116 {'preprocessing__numerical__imputer__strategy': 'mean', 'preprocessing__numerical__poly__degree': 3, 'preprocessing__numerical__poly__include_bias': False, 'preprocessing__numerical__poly__interaction_only': False}
206771298.9352535 {'preprocessing__numerical__imputer__strategy': 'mean', 'preprocessing__numerical__poly__degree': 3, 'preprocessing__numerical__poly__include_bias': True, 'preprocessing__numerical__poly__interaction_only': False}
94247368586.192 {'preprocessing__numerical__imputer__strategy': 'mean'

In [ ]:
grid_search_scenario_2.best_params_

{'preprocessing__numerical__imputer__strategy': 'mean',
 'preprocessing__numerical__poly__degree': 2,
 'preprocessing__numerical__poly__include_bias': False,
 'preprocessing__numerical__poly__interaction_only': False}

In [ ]:
# if refit=True, it returns a model trained with the full training set with the best hyperparameters
best_model = grid_search_scenario_2.best_estimator_
best_model

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('feat_engineering',
                                                                   HousingFeatEngineering()),
                                                                  ('drop_features',
                                                                   DropFeatures()),
                                                                  ('poly',
                                                                   PolynomialFeatures(include_bias=False)),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                             

In [ ]:
# best approximated RMSE
np.sqrt(-grid_search_scenario_2.best_score_)

626165.2300283933

In [ ]:
# get the approximated RMSE and its standard deviation
best_index = grid_search_scenario_2.best_index_
best_score = np.sqrt(-grid_search_scenario_2.cv_results_['mean_test_score'][best_index])
best_score_std = np.sqrt(grid_search_scenario_2.cv_results_['std_test_score'][best_index])

print(f'Best score: {best_score} +- {best_score_std}')

Best score: 626165.2300283933 +- 874997.3364957358


In [ ]:
n_folds = 5
split_keys = [f'split{i}_test_score' for i in range(n_folds)]
best_index = grid_search_scenario_2.best_index_

rmse_scores = []

for key in split_keys:
    neg_mse_score = grid_search_scenario_2.cv_results_[key][best_index]
    rmse_scores.append(np.sqrt(-neg_mse_score))

best_rmse = np.mean(rmse_scores)
best_rmse_std = np.std(rmse_scores)


print(f'Best RMSE score: {best_rmse} +- {best_rmse_std}')

Best RMSE score: 345172.7167522515 +- 522435.34614953154


<br/>

The large **best true RMSE** (\\$345,172.71) -- even larger for the approximated one (\\$626,165.23) -- shows that (i) this model is not suitable for the problem, and/or (ii) the considered _search space_ is insufficient to get better models. <br/>

We'd better throw this model away.